<a href="https://colab.research.google.com/github/Tucker6742/Segment_testing/blob/main/GroundingDino_SAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision
!pip install -U git+https://github.com/luca-medeiros/lang-segment-anything.git

  Cloning https://github.com/luca-medeiros/lang-segment-anything.git to /tmp/pip-req-build-4wvqdklp
  Running command git clone --filter=blob:none --quiet https://github.com/luca-medeiros/lang-segment-anything.git /tmp/pip-req-build-4wvqdklp
  Resolved https://github.com/luca-medeiros/lang-segment-anything.git to commit fcdb92f9ae4ac3a5b4e5fd85c1576e09132f10bc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/IDEA-Research/GroundingDINO.git to /tmp/pip-install-cp0txkwk/groundingdino_8b3f322271ec4c5fb0fc62581a6d2154
  Running command git clone --filter=blob:none --quiet https://github.com/IDEA-Research/GroundingDINO.git /tmp/pip-install-cp0txkwk/groundingdino_8b3f322271ec4c5fb0fc62581a6d2154
  Resolved https://github.com/IDEA-Research/GroundingDINO.git to commit 60d796825e1266e56f7e4e9e00e88de662b67bd3
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/fac

In [ ]:
from PIL import Image
from lang_sam import LangSAM
import torch
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

# GroundingDino to cut out rice seed

In [ ]:
def save_mask(mask_np, filename):
    mask_image = Image.fromarray((mask_np * 255).astype(np.uint8))
    mask_image.save(filename)

def display_image_with_masks(image, masks):
    num_masks = len(masks)

    fig, axes = plt.subplots(1, num_masks + 1, figsize=(15, 5))
    axes[0].imshow(image)
    axes[0].set_title("Original Image")
    axes[0].axis('off')

    for i, mask_np in enumerate(masks):
        axes[i+1].imshow(mask_np, cmap='gray')
        axes[i+1].set_title(f"Mask {i+1}")
        axes[i+1].axis('off')

    plt.tight_layout()
    plt.show()

def display_image_with_boxes(image, boxes, logits):
    fig, ax = plt.subplots()
    ax.imshow(image)
    ax.set_title("Image with Bounding Boxes")
    ax.axis('off')

    for box, logit in zip(boxes, logits):
        x_min, y_min, x_max, y_max = box
        confidence_score = round(logit.item(), 2)  # Convert logit to a scalar before rounding
        box_width = x_max - x_min
        box_height = y_max - y_min

        # Draw bounding box
        rect = plt.Rectangle((x_min, y_min), box_width, box_height, fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)

        # Add confidence score as text
        ax.text(x_min, y_min, f"Confidence: {confidence_score}", fontsize=8, color='red', verticalalignment='top')

    plt.show()

def print_bounding_boxes(boxes):
    print("Bounding Boxes:")
    for i, box in enumerate(boxes):
        print(f"Box {i+1}: {box}")

def print_detected_phrases(phrases):
    print("\nDetected Phrases:")
    for i, phrase in enumerate(phrases):
        print(f"Phrase {i+1}: {phrase}")

def print_logits(logits):
    print("\nConfidence:")
    for i, logit in enumerate(logits):
        print(f"Logit {i+1}: {logit}")



In [ ]:
size = (500, 500)

image_pil = Image.open("/content/gdrive/MyDrive/Colab Notebooks/Son_prj/test_img/pics/G9.JPG").convert("RGB")
image_pil.thumbnail(size,Image.ANTIALIAS)
model = LangSAM()

In [ ]:
text_prompt = "rice seed"
masks, boxes, phrases, logits = model.predict(image_pil, text_prompt)

if len(masks) == 0:
    print(f"No objects of the '{text_prompt}' prompt detected in the image.")
else:
    # Convert masks to numpy arrays
    masks_np = [mask.squeeze().cpu().numpy() for mask in masks]

    # Display the original image and masks side by side
    display_image_with_masks(image_pil, masks_np)

    # Display the image with bounding boxes and confidence scores
    display_image_with_boxes(image_pil, boxes, logits)

    # Save the masks
    for i, mask_np in enumerate(masks_np):
        mask_path = f"/content/gdrive/MyDrive/Colab Notebooks/Son_prj/test_img/mask/image_mask_{i+1}.png"
        save_mask(mask_np, mask_path)

    # Print the bounding boxes, phrases, and logits
    print_bounding_boxes(boxes)
    print_detected_phrases(phrases)
    print_logits(logits)


In [ ]:
mask_path = '/content/gdrive/MyDrive/Colab Notebooks/Son_prj/test_img/mask'
mask_files = os.listdir(mask_path)
masks = []
for mask_file in mask_files:
    masks.append(np.array(Image.open(mask_path + '/' + mask_file)))


In [ ]:
final_mask = np.array(sum(masks))

In [ ]:
final_mask.shape

In [ ]:
img = np.array(image_pil)
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        img[i][j] = img[i][j] * final_mask[i][j]
plt.imshow(img)

# SAM to segment each rice seed

In [ ]:
import sys
import cv2
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
sam_checkpoint = "/root/.cache/torch/hub/checkpoints/sam_vit_h_4b8939.pth"
model_type = "vit_h"

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

In [ ]:
mask_generator = SamAutomaticMaskGenerator(sam)
#     model=sam,
#     points_per_side=32,
#     pred_iou_thresh=0.86,
#     stability_score_thresh=0.92,
#     box_nms_thresh = 0.9,
#     crop_n_layers=1,
#     crop_n_points_downscale_factor=2,
#     min_mask_region_area=100,  # Requires open-cv to run post-processing
# )

In [ ]:
masks = mask_generator.generate(img)

In [ ]:
import matplotlib.patches as patches
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(True)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0


    for ann in sorted_anns:
        m = ann['segmentation'] #[[F,T,F], [T,F,],...]
        b = ann['bbox']

        #cv2.rectangle(img, (b[0], b[1]), (b[0]+b[2], b[1]+b[3]), (0,255,0),2)

        # Create a rectangle patch
        rect = patches.Rectangle((b[0], b[1]), b[2], b[3], linewidth=2, edgecolor='g', facecolor='none')
        # Add the rectangle patch to the plot
        ax.add_patch(rect)

        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask

    ax.imshow(img)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img)
show_anns(masks)
plt.axis('off')
plt.show()

In [ ]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

In [ ]:
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))

In [ ]:
mask_2 = SamPredictor(sam)
mask_2.set_image(img)

In [ ]:
bbox = [np.array(mask['bbox']) for mask in masks]
area = []
for b in bbox:
    area.append(b[2]*b[3])
plt.hist(area, bins = 100)
plt.show()

In [ ]:
area_chosen = [a for a in area if a < 1000]
for a in area_chosen:

    chosen_index = area.index(a)
    box = bbox[chosen_index]
    input_box = np.ones((4, ))
    # Change input_box to XYXY from XYWH
    input_box[0] = box[0]
    input_box[1] = box[1]
    input_box[2] = box[0] + box[2]
    input_box[3] = box[1] + box[3]

    mask, _, _ = mask_2.predict(
        point_coords=None,
        point_labels=None,
        box=input_box[None, :],
        multimask_output=False,)
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    show_mask(mask[0], plt.gca())
    show_box(input_box, plt.gca())
    plt.axis('off')
    plt.show()

In [ ]:
len(area_chosen)